In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

matplotlib.rcParams.update({'figure.figsize': (15, 9)})
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams.update({'axes.labelsize': 20})
matplotlib.rcParams.update({'xtick.labelsize': 12})
matplotlib.rcParams.update({'ytick.labelsize': 12})
matplotlib.rcParams.update({'font.family': 'Helvetica, Arial, sans-serif'})
%config InlineBackend.figure_format = 'retina'

sns.set_style("whitegrid")

# Machine Learning lab 4: extending logistic regression
## Jake Rowland and Paul Herz
2017-10-01

## 1. Introduction

TODO

### 1.1 Background

TODO

### 1.2 Business Case

TODO

### 1.3 Serviceability

TODO

## 2. The dataset: preprocessing and review

### 2.1 Dataset preparation

TODO

### 2.2 Data quality

TODO

## 3. Something



In [2]:
import pandas as pd

m = pd.read_csv('movie_metadata.csv')

# Reorder the DataFrame to a more intelligent fashion
m = m[[
    'movie_title','title_year',
    'genres', 'plot_keywords', 'duration',
    'budget', 'gross',
    'language', 'country', 'content_rating',
    'color', 'aspect_ratio',
    'facenumber_in_poster',
    'director_name',
    'actor_1_name', 'actor_2_name', 'actor_3_name',
    'movie_facebook_likes', 'director_facebook_likes', 'actor_1_facebook_likes', 'actor_2_facebook_likes',
    'actor_3_facebook_likes', 'cast_total_facebook_likes',
    'movie_imdb_link', 'num_user_for_reviews', 'num_critic_for_reviews', 'num_voted_users',
    'imdb_score',
]]

# Reduce the number of float64 data types for columns that do not need a float64 data type
for col in ['title_year','facenumber_in_poster',
'movie_facebook_likes','actor_1_facebook_likes','actor_2_facebook_likes',
'actor_3_facebook_likes','cast_total_facebook_likes','num_user_for_reviews',
'num_critic_for_reviews','num_voted_users']:
    m[col] = pd.to_numeric(m[col],downcast='integer')
  
# Remove all duplicate entries
m.drop_duplicates(inplace=True)

# Create a copy to perserve the original DataFrame
m_original=m.copy()
m.head()

,movie_title,title_year,genres,plot_keywords,duration,budget,gross,language,country,content_rating,...,director_facebook_likes,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,cast_total_facebook_likes,movie_imdb_link,num_user_for_reviews,num_critic_for_reviews,num_voted_users,imdb_score
0,Avatar,2009.0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,178.0,237000000.0,760505847.0,English,USA,PG-13,...,0.0,1000.0,936.0,855.0,4834,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,723.0,886204,7.9
1,Pirates of the Caribbean: At World's End,2007.0,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,169.0,300000000.0,309404152.0,English,USA,PG-13,...,563.0,40000.0,5000.0,1000.0,48350,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,302.0,471220,7.1
2,Spectre,2015.0,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,148.0,245000000.0,200074175.0,English,UK,PG-13,...,0.0,11000.0,393.0,161.0,11700,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,602.0,275868,6.8
3,The Dark Knight Rises,2012.0,Action|Thriller,deception|imprisonment|lawlessness|police offi...,164.0,250000000.0,448130642.0,English,USA,PG-13,...,22000.0,27000.0,23000.0,23000.0,106759,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,813.0,1144337,8.5
4,Star Wars: Episode VII - The Force Awakens ...,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,131.0,131.0,12.0,NaN,143,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,8,7.1


In [3]:
# Remove foreign films to solve the normalization problem
m = m[m['country'] == 'USA']

# Remove items with non-American or non-film rating systems
m = m[m['content_rating'].isin(['R','PG-13','PG','G'])]

In [4]:
# Categorize the IMDB score into three classes:
# [0%-49%] is Poor, [50%-89%] is Average, [90%-100%] is Good.
poor_avg = m['imdb_score'].quantile(.5)
avg_good = m['imdb_score'].quantile(.9)
m['rating_category'] = pd.cut(m.imdb_score,[0,poor_avg,avg_good,10],labels=['poor','average','good'])

In [5]:
for c in ['movie_title','plot_keywords','actor_1_name','actor_2_name','actor_3_name',
'movie_imdb_link','genres', 'director_name','imdb_score','aspect_ratio','country','language']:
    m.drop(c, axis=1, inplace=True)

In [6]:
# Convert categorical values to category type
for col in ['content_rating','color']:
    m[col] = m[col].astype('category')

In [7]:
# Remove rows where the value is null
for col in ['title_year', 'language','country','content_rating',
'aspect_ratio','duration', 'color','gross','budget','movie_facebook_likes',
'actor_1_facebook_likes','actor_2_facebook_likes','actor_3_facebook_likes',
'cast_total_facebook_likes']:
    try:
        m = m[pd.notnull(m[col])]
    except KeyError:
        pass

In [8]:
# Assume null review counts are 0
for col in ['num_user_for_reviews','num_critic_for_reviews']:
    m[col].fillna(value=0,inplace=True)

In [9]:
# Assume missing face counts are the mean
avgFace = round(m['facenumber_in_poster'].mean())
m['facenumber_in_poster'].fillna(value=avgFace, inplace=True)

In [10]:
# replace color and content_rating with dummies
m = pd.get_dummies(m, columns=['color','content_rating'])
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2963 entries, 0 to 5042
Data columns (total 21 columns):
title_year                   2963 non-null float64
duration                     2963 non-null float64
budget                       2963 non-null float64
gross                        2963 non-null float64
facenumber_in_poster         2963 non-null float64
movie_facebook_likes         2963 non-null int32
director_facebook_likes      2963 non-null float64
actor_1_facebook_likes       2963 non-null float64
actor_2_facebook_likes       2963 non-null float64
actor_3_facebook_likes       2963 non-null float64
cast_total_facebook_likes    2963 non-null int32
num_user_for_reviews         2963 non-null float64
num_critic_for_reviews       2963 non-null float64
num_voted_users              2963 non-null int32
rating_category              2963 non-null category
color_ Black and White       2963 non-null uint8
color_Color                  2963 non-null uint8
content_rating_G             2963 n

In [25]:
if m.rating_category.dtype != np.dtype('int8'):
    m.rating_category = m.rating_category.cat.codes
m.rating_category.value_counts()

0    1456
1    1231
2     276
Name: rating_category, dtype: int64

In [39]:
X=m.drop('rating_category', axis=1, inplace=False)
y=np.ravel(m['rating_category'])

split_index = int(len(m)*0.8)
X_train = X[:split_index]
X_predict = X[split_index:]
y_train = y[:split_index]
y_predict = y[split_index:]

In [41]:
logistic = linear_model.LogisticRegression(multi_class='ovr')
logistic.fit(X_train,y_train)
C = logistic.predict(X_predict)
print(pd.Series(C).value_counts())
print(pd.Series(y_predict).value_counts())

1    369
0    200
2     24
dtype: int64
1    285
0    258
2     50
dtype: int64
